In [3]:
def get_jobs(keyword, num_jobs, verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
    from selenium import webdriver
    import time
    import pandas as pd
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="E://chromedriver", options=options)
    driver.set_window_size(1120, 1000)
    
    #url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword="+keyword+"&locT=&locId=&jobType="
    url='https://www.glassdoor.co.in/Job/india-data-scientist-jobs-SRCH_IL.0,5_IN115_KE6,20.htm'
    #url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(15)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_class_name("ModalStyle__xBtn___29PT9").click()  #clicking to the X.
        except NoSuchElementException:
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [4]:
import pandas as pd
#import glassdoor_scraper as gs

In [5]:
#driver = webdriver.Chrome(executable_path=r'E://Board_Infinity//chromedriver.exe')
df=get_jobs('data scientist',9006
            ,False)

Progress: 0/900
Progress: 1/900
Progress: 2/900
Progress: 3/900
Progress: 4/900
Progress: 5/900
Progress: 6/900
Progress: 7/900
Progress: 8/900
Progress: 9/900
Progress: 10/900
Progress: 11/900
Progress: 12/900
Progress: 13/900
Progress: 14/900
Progress: 15/900
Progress: 16/900
Progress: 17/900
Progress: 18/900
Progress: 19/900
Progress: 20/900
Progress: 21/900
Progress: 22/900
Progress: 23/900
Progress: 24/900
Progress: 25/900
Progress: 26/900
Progress: 27/900
Progress: 28/900
Progress: 29/900
Progress: 30/900
Progress: 31/900
Progress: 32/900
Progress: 33/900
Progress: 34/900
Progress: 35/900
Progress: 36/900
Progress: 37/900
Progress: 38/900
Progress: 39/900
Progress: 40/900
Progress: 41/900
Progress: 42/900
Progress: 43/900
Progress: 44/900
Progress: 45/900
Progress: 46/900
Progress: 47/900
Progress: 48/900
Progress: 49/900
Progress: 50/900
Progress: 51/900
Progress: 52/900
Progress: 53/900
Progress: 54/900
Progress: 55/900
Progress: 56/900
Progress: 57/900
Progress: 58/900
Progres

Progress: 462/900
Progress: 463/900
Progress: 464/900
Progress: 465/900
Progress: 466/900
Progress: 467/900
Progress: 468/900
Progress: 469/900
Progress: 470/900
Progress: 471/900
Progress: 472/900
Progress: 473/900
Progress: 474/900
Progress: 475/900
Progress: 476/900
Progress: 477/900
Progress: 478/900
Progress: 479/900
Progress: 480/900
Progress: 481/900
Progress: 482/900
Progress: 483/900
Progress: 484/900
Progress: 485/900
Progress: 486/900
Progress: 487/900
Progress: 488/900
Progress: 489/900
Progress: 490/900
Progress: 491/900
Progress: 492/900
Progress: 493/900
Progress: 494/900
Progress: 495/900
Progress: 496/900
Progress: 497/900
Progress: 498/900
Progress: 499/900
Progress: 500/900
Progress: 501/900
Progress: 502/900
Progress: 503/900
Progress: 504/900
Progress: 505/900
Progress: 506/900
Progress: 507/900
Progress: 508/900
Progress: 509/900
Progress: 510/900
Progress: 511/900
Progress: 512/900
Progress: 513/900
Progress: 514/900
Progress: 515/900
Progress: 516/900
Progress: 

In [6]:
df.to_csv("jobs_dataset.csv")

In [7]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Scientist,-1,Site Name: India - Karnataka - Bangalore\nPost...,3.9,GSK\n3.9,Bengaluru,"Brentford, United Kingdom",10000+ employees,1830,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,₹500+ billion (INR),"Pfizer, AstraZeneca, Merck"
1,Data Scientist,-1,Overview\n\n\nWe have an exciting opportunity ...,4.0,Quanticate\n4.0,Bengaluru,"Hitchin, United Kingdom",201 to 500 employees,1995,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,₹1 to ₹5 billion (INR),"GCE Solutions, Parexel, IQVIA"
2,Data Scientist,-1,Provide analytical insights into emerging prob...,3.5,PayPal\n3.5,Bengaluru,"San Jose, CA",10000+ employees,1998,Company - Public,Internet,Information Technology,₹500+ billion (INR),"Square, Amazon, Apple"
3,Data Scientist,-1,"At Amazon, we strive to be most customer-centr...",4.2,Amazon\n4.2,Hyderabad,"Seattle, WA",10000+ employees,1994,Company - Public,Internet,Information Technology,₹500+ billion (INR),"Google, Microsoft, Walmart"
4,Data Scientist,-1,"When everything's connected, how we connect is...",3.3,TTEC\n3.3,Hyderabad,"Englewood, CO",10000+ employees,1982,Company - Public,Staffing & Outsourcing,Business Services,₹100 to ₹500 billion (INR),"Teleperformance, TaskUs, Convergys"
